# Pythia 70 M 


## Find a Next Token Head


In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/mats_sae_training_pythia_70M_hook_q_L2H7/sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_1024:v16', type='model')
artifact_dir = artifact.download()

In [ ]:

model = HookedTransformer.from_pretrained(
    "pythia-70m-deduped",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)


# path = "../checkpoints/denvkhqv/final_sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_256.pt"
# path = "./artifacts/sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_512:v0/final_sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_512.pt"
# path = "../checkpoints/c0kldy9v/final_sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_1024.pt"
# path = "../checkpoints/m6x4lz0v/600002560_sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_1024.pt"
path = "./artifacts/sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_1024:v16/final_sparse_autoencoder_pythia-70m-deduped_blocks.2.attn.hook_q_1024.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

In [ ]:

# Test on individual prompt (random repeating tokens)
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=16, n_repeat_tokens=2)
prompt = model.to_string(random_tokens)
print(prompt)

(logits, loss), cache = model.run_with_cache(prompt, return_type = "both", loss_per_token = True)
px.line(loss.squeeze().detach().cpu().numpy()).show()

from transformer_lens.head_detector import *

model.to("cpu")
pattern = get_previous_token_head_detection_pattern(model.to_tokens(prompt))
pattern = pattern.to("cpu")
px.imshow(detect_head(model, prompt, pattern), title="Previous token head detection").show()
# pattern = get_duplicate_token_head_detection_pattern(model.to_tokens(prompt).to("cpu"))
# pattern = pattern.to("cpu")
# px.imshow(detect_head(model, prompt, pattern), title="Duplicate token head detection").show()
# pattern = get_induction_head_detection_pattern(model.to_tokens(prompt).to("cpu"))
# pattern = pattern.to("cpu")
# px.imshow(detect_head(model, prompt, pattern), title="Duplicate token head detection").show()
model = model.to("mps")

In [ ]:
px.imshow(
    cache["blocks.2.attn.hook_pattern"][0].detach().cpu(),
    facet_col=0,
    facet_col_wrap=8,
).show()

In [73]:
random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=16, n_repeat_tokens=2)
prompt = model.to_string(random_tokens)
token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt], model, sparse_autoencoder)
print(token_df.columns)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm",
)

Index(['str_tokens', 'unique_token', 'context', 'batch', 'pos', 'label',
       'loss', 'max_idx_pos', 'max_idx_tok', 'max_idx_tok_value',
       'top10_token_suppression_diffs', 'top10_token_suppression_inds',
       'top10_token_boosting_vals', 'top10_token_boosting_inds',
       'ablated_loss', 'loss_diff', 'q_norm', 'rec_q_norm', 'mse_loss',
       'explained_variance', 'num_active_features', 'top_k_feature_acts',
       'top_k_features', 'rec_q_max_idx_pos', 'rec_q_max_idx_tok',
       'rec_q_max_idx_tok_value', 'kl_divergence'],
      dtype='object')


,str_tokens,unique_token,context,batch,pos,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence
0,John,John/0,| John| plaintiffs refugeeAdv Iranians,0,0,0/0,14.660598,-0.084303,5.684970,3,0.042869,0.000000
1,plaintiffs,plaintiffs/1,John| plaintiffs| refugeeAdv Iranianscal,0,1,0/1,11.215460,0.181187,11.857803,7,0.065929,0.005066
2,refugee,refugee/2,John plaintiffs| refugee|Adv Iranianscalak,0,2,0/2,12.493197,-0.384398,8.779474,2,0.061659,0.008420
3,Adv,Adv/3,John plaintiffs refugee|Adv| Iranianscalakv,0,3,0/3,14.381540,0.013397,8.989880,4,0.056237,0.006026
4,Iran,Iran/4,John plaintiffs refugeeAdv| Iran|ianscalakv支,0,4,0/4,16.540043,-0.379021,6.782185,1,0.049993,0.000819
5,ians,ians/5,John plaintiffs refugeeAdv Iran|ians|calakv支 Full,0,5,0/5,5.432947,-0.694098,12.113808,3,0.078807,0.003770
6,cal,cal/6,John plaintiffs refugeeAdv Iranians|cal|akv支 Full volumes,0,6,0/6,13.230392,-0.193801,12.702828,7,0.067352,0.003615
7,ak,ak/7,John plaintiffs refugeeAdv Iranianscal|ak|v支 Full volumes argv,0,7,0/7,7.887805,0.057912,11.755912,5,0.057637,0.007611
8,v,v/8,John plaintiffs refugeeAdv Iranianscalak|v|支 Full volumes argvund,0,8,0/8,10.428738,0.608473,10.007791,6,0.044834,0.002956
9,支,支/9,John plaintiffs refugeeAdv Iranianscalakv|支| Full volumes argvunddefendants,0,9,0/9,16.512814,0.459944,11.779767,5,0.062713,0.008861


In [ ]:
HEAD_IDX = sparse_autoencoder.cfg.hook_point_head_index
LAYER_IDX = sparse_autoencoder.cfg.hook_point_layer
patterns_original = original_cache[utils.get_act_name("attn_scores", LAYER_IDX)][0,head_index].detach().cpu()
patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("attn_scores", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
both_patterns = torch.stack([patterns_original, patterns_reconstructed])
plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")
# patterns_original = original_cache[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
# patterns_reconstructed = cache_reconstructed_query[utils.get_act_name("pattern", LAYER_IDX)][0,HEAD_IDX].detach().cpu()
# both_patterns = torch.stack([patterns_original, patterns_reconstructed])
# plot_attn(both_patterns.detach().cpu(), token_df, title="Original and Reconstructed Attention Distribution")

In [ ]:
for i in range(5,8):
    print(token_df["unique_token"][i])
    plot_line_with_top_10_labels(feature_acts[i], "", 3)

In [ ]:
POS_INTEREST = feature_acts.shape[0] - 1 # index from 0.

# reload(joseph.visualisation)
# from joseph.visualisation import *

print(token_df["unique_token"][POS_INTEREST]) # bos gone.
plot_line_with_top_10_labels(feature_acts[POS_INTEREST], "", 3)

# vals, inds = torch.topk(feature_acts[POS_INTEREST],5)

# print(inds)
# plot_attn_score_by_feature(model, sparse_autoencoder, inds, original_cache, token_df, pos_interest=POS_INTEREST, vals = vals)
# plot_unembed_score_by_feature(model, sparse_autoencoder, inds, token_df, vals=vals)

In [ ]:
# since we can't use linear analysis when we have rotary embeddings, we should use causal interventions. 

# Analysis over Training Distribution

In [ ]:
data = get_webtext()

In [ ]:
str_token_list = []
loss_list = []
ablated_loss_list = []
# data = get_webtext()

NUM_PROMPTS = 10
# MAX_PROMPT_LEN = 100
# BATCH_SIZE = 10
dataframe_list = []
feature_acts_list = []
with torch.no_grad():
    for i in tqdm(range(NUM_PROMPTS)):
        
        # Get Token Data
        prompt = data[i]
        # new_str = data[BATCH_SIZE * i: BATCH_SIZE * (i + 1)]
        

        token_df, _, _, feature_acts = eval_prompt(prompt, model, sparse_autoencoder)
        feature_acts_list.append(feature_acts)
        dataframe_list.append(token_df)
        
all_token_df = pd.concat(dataframe_list)
all_token_df.reset_index(drop=True)
all_token_features = torch.cat(feature_acts_list)

print(all_token_df.shape)
print(all_token_df.columns)
all_token_df.head()

In [ ]:
pos_mask = (all_token_df.pos.values < 128)
all_token_df = all_token_df[all_token_df.pos < 128]
all_token_features = all_token_features[torch.tensor(pos_mask)]

In [ ]:
all_token_features.sum(dim=0).nonzero().shape

In [ ]:
all_token_df.shape

In [ ]:
sparsity = (all_token_features > 0).float().mean(dim=0).detach().cpu()
log_sparsity = torch.log10(sparsity)
print((sparsity > 0).float().mean())

In [ ]:
px.histogram(log_sparsity, nbins=100).show()
px.strip(log_sparsity, hover_data=[[i for i in range(sparse_autoencoder.cfg.d_sae)]]).show()

In [ ]:
all_token_df.pos = all_token_df.pos.astype(str)
# get some random features
random_features = torch.randint(0, 1024, (3,))
for feature in torch.tensor([560,773]):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    feature_density = (all_token_df[f"feature_{feature}"] > 0).mean()
    if feature_density > 0:
        print(f"Feature {feature} has density {feature_density}")
        px.scatter(
            all_token_df,
            x = "pos",
            y = f"feature_{feature}",
            color="pos",
            hover_data=["unique_token", "context", "label"]
            ).show()


# Exploratory

In [68]:
position = 35
mask = [i  % 128 == position  for i in range(all_token_df.shape[0])]
scores = (all_token_features[mask, :] > 0).float().mean(dim=0)
vals, inds = torch.topk(scores, 10)
all_token_df[mask].head()


,str_tokens,unique_token,context,batch,pos,label,loss,max_idx_pos,max_idx_tok,max_idx_tok_value,...,rec_q_norm,mse_loss,explained_variance,num_active_features,top_k_feature_acts,top_k_features,rec_q_max_idx_pos,rec_q_max_idx_tok,rec_q_max_idx_tok_value,kl_divergence
35,commissioner,commissioner/35,ell will spend the 10th anniversary of being n...,0,35,0/35,8.367705,35,named,0.795457,...,10.466471,6.769562,0.058970,4,"[0.8071162700653076, 0.6404762268066406, 0.161...","[700, 324, 179, 9, 0, 1, 2, 3, 4, 5]",35,named,0.821393,0.018114
35,ball,ball/35,'s one week left to register for the Adult Sof...,0,35,0/35,0.750651,35,Soft,0.943761,...,11.353163,8.017096,0.054488,6,"[0.4053466320037842, 0.23056399822235107, 0.22...","[268, 412, 488, 115, 848, 48, 0, 1, 2, 3]",35,Soft,0.920011,0.004721
35,rockets,rockets/35,after Palestinian militants fired more than 4...,0,35,0/35,0.169942,35,am,0.710706,...,10.811336,10.561659,0.084869,4,"[0.43842077255249023, 0.19832146167755127, 0.1...","[42, 534, 78, 545, 0, 1, 2, 3, 4, 5]",35,am,0.782510,0.024647
35,be,be/35,club than anyone in their history. But he wil...,0,35,0/35,1.891442,35,will,0.769866,...,10.849177,5.468954,0.044782,3,"[1.2411943674087524, 0.1936246156692505, 0.051...","[804, 339, 770, 0, 1, 2, 3, 4, 5, 6]",35,will,0.792280,0.005982
35,\n,\n/35,"the Photo Collection, Los Angeles Public Libr...",0,35,0/35,0.041990,35,\n,0.474263,...,10.370700,3.347119,0.029734,3,"[2.5998079776763916, 1.3701636791229248, 0.625...","[292, 343, 725, 0, 1, 2, 3, 4, 5, 6]",35,\n,0.537760,0.010179


In [69]:
plot_line_with_top_10_labels(scores, "", 10)
plot_line_with_top_10_labels(all_token_features[position,], "", 3)
plot_line_with_top_10_labels(all_token_features[position+128,], "", 3)
plot_line_with_top_10_labels(all_token_features[position+128*2,], "", 3)

In [71]:
filter_columns = ["pos", "unique_token", "context", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
for feature in torch.tensor(inds):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")

115 0.034375


,pos,unique_token,context,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence,feature_115
35,35,ast/35,free-agent defensive tackle Terrance “Pot Ro|...,0/35,0.009756,1.437014,6.496494,9,0.033652,0.003060,1.479548
97,97,ino/97,", the Democratic nominee for lieutenant govern...",0/97,0.000236,0.403348,9.533668,11,0.048979,0.002664,1.433475
86,86,ball/86,can sign up to play at the Byers Soft|ball| C...,0/86,0.007020,0.693337,9.143553,4,0.052455,0.006847,1.260757
98,98,za/98,the Democratic nominee for lieutenant governo...,0/98,0.002781,4.673353,6.238575,4,0.036321,0.009673,1.213840
108,108,anda/108,’s business from the beginning. The first P|an...,0/108,0.002113,5.938484,9.558233,5,0.054650,0.003653,1.192723


268 0.021875


,pos,unique_token,context,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence,feature_268
42,42,ger/42,"Library.\n\nNow fifty years old, Dod|ger| Sta...",0/42,0.893382,0.135251,8.036793,9,0.052917,0.014654,0.802116
9,9,ocratic/9,"Nearly six years ago, then-Dem|ocratic| Nation...",0/9,1.652764,-0.151435,11.597414,10,0.091844,0.076073,0.761337
27,27,agent/27,"be a natural fit for the Raiders, free-|agent...",0/27,0.921944,1.604017,10.493539,10,0.070294,0.051580,0.585264
35,35,ast/35,free-agent defensive tackle Terrance “Pot Ro|...,0/35,0.009756,1.437014,6.496494,9,0.033652,0.003060,0.498041
35,35,ball/35,'s one week left to register for the Adult Sof...,0/35,0.750651,0.325829,8.017096,6,0.054488,0.004721,0.405347


292 0.01484375


...

,pos,unique_token,context,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence,feature_848
6,6,ast/6,Turns out the Pot Ro|ast| was undercooked.,0/6,0.777187,0.525970,9.600437,8,0.049564,0.002970,1.280623
85,85,top/85,the real estate possibilities associated with...,0/85,1.626816,-0.029188,9.007072,4,0.059097,0.011376,1.230012
78,78,s/78,"to Caltech.\n\nFor the Cherng|s|, the decisio...",0/78,6.204171,-3.788802,10.805756,6,0.067599,0.012984,1.088837
61,61,on/61,", 2006) clunking heads together in Cant|on| an...",0/61,0.247958,0.164134,11.400770,7,0.063143,0.013232,0.898249
35,35,ast/35,free-agent defensive tackle Terrance “Pot Ro|...,0/35,0.009756,1.437014,6.496494,9,0.033652,0.003060,0.825998


9 0.03828125


,pos,unique_token,context,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence,feature_9
80,80,people/80,near daily occurrence but have only occasiona...,0/80,3.722107,-0.719109,6.173653,1,0.054815,0.018072,1.380925
54,54,people/54,to prepare for the future\n\n“I think| people...,0/54,5.713583,0.237531,5.928537,4,0.050611,0.004312,1.298762
45,45,farmers/45,upon common recommendations for the developme...,0/45,1.610658,0.416249,6.784739,3,0.058192,0.007244,1.255878
99,99,athletes/99,reminding us that before the land served as h...,0/99,8.685326,-0.009255,3.012216,4,0.029737,0.019055,1.244000
64,64,farmers/64,Dialogue also hopes to identify the key chall...,0/64,0.592093,0.410905,7.286697,5,0.061809,0.010268,1.242444


42 0.02421875


,pos,unique_token,context,label,loss,loss_diff,mse_loss,num_active_features,explained_variance,kl_divergence,feature_42
117,117,child/117,the sources said. A six-month-old| child| was...,0/117,3.889221,-0.305498,5.139321,6,0.045979,0.030866,1.125435
66,66,melanoma/66,found that can delay progression of metastati...,0/66,0.000323,0.000251,7.820819,4,0.062497,0.003905,0.719197
57,57,heads/57,"on Aug. 8, 2006) clunking| heads| together in...",0/57,8.107248,0.006687,7.133803,3,0.055069,0.013392,0.602288
49,49,landmark/49,"old, Dodger Stadium has become a Los Angeles|...",0/49,4.874801,-0.087040,6.936670,5,0.066104,0.010682,0.558756
118,118,community/118,", it was the site of a thriving Mexican Americ...",0/118,0.606771,0.242754,5.991634,9,0.053587,0.034860,0.557661


# Causal

In [98]:
def test_get_attn_sae_feature_removal(example_prompt, 
                                      example_prompt_answer, 
                                      model, 
                                      sparse_autoencoder,
                                      features_to_remove=[0],
                                      token_unembed_to_remove=[]):
    
    layer_index = sparse_autoencoder.cfg.hook_point_layer
    head_index = sparse_autoencoder.cfg.hook_point_head_index
    
    example_prompt_tokens = model.to_tokens(example_prompt)
    attn_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([example_prompt], model, sparse_autoencoder)

    original_acts = original_cache[sparse_autoencoder.cfg.hook_point]
    print(original_acts.shape)
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
       original_acts[0,:,head_index]
    )
    

    def remove_feature_hook(hook_in, hook, head = head_index, features_to_remove = features_to_remove):
        for feature_to_remove in features_to_remove:
            feature_dir = feature_acts[-1,feature_to_remove]*sparse_autoencoder.W_dec[feature_to_remove]
            hook_in[:, :, head] -= feature_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_feature_hook)]):
        _, cache_removed_feature = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        
        
    def replace_with_feature_hook(hook_in, hook, head = head_index, features_to_add = features_to_remove):
        new_act = torch.zeros_like(hook_in[:, :, head])
        for features_to_add in features_to_remove:
            feature_dir = feature_acts[-1,features_to_add]*sparse_autoencoder.W_dec[features_to_add]
            new_act += feature_dir
        hook_in[:, :, head] = new_act
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, replace_with_feature_hook)]):
        _, cache_only_features = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)
        

    def remove_token_unembed_hook(hook_in, hook, head = head_index, token_unembed_to_remove = token_unembed_to_remove):
        for token_id in token_unembed_to_remove:
            unembed_dir = model.W_U[:, token_id] @ model.W_Q[LAYER_IDX,head_index]
            unembed_dir_proj =  hook_in[:, :, head] @ unembed_dir.T
            hook_in[:, :, head] -= unembed_dir_proj[:,:,None]*unembed_dir
        return hook_in
    
    with model.hooks(fwd_hooks=[(sparse_autoencoder.cfg.hook_point, remove_token_unembed_hook)]):
        _, cache_removed_token_unembed = model.run_with_cache(example_prompt_tokens, return_type="loss", loss_per_token=True)

    patterns = original_cache[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df = make_token_df(model, example_prompt_tokens)
    # remove last row of attn df
    attn_df["original_attn"] = patterns[-1,]
    patterns = cache_reconstructed_query[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["reconstructed_attn"] = patterns[-1,]
    patterns = cache_removed_feature[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["ablated_feature_attn"] = patterns[-1,]
    patterns = cache_only_features[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["cache_only_features"] = patterns[-1,]
    patterns = cache_removed_token_unembed[f"blocks.{layer_index}.attn.hook_pattern"][0,head_index].detach().cpu()
    attn_df["cache_removed_token_unembed"] = patterns[-1,]
    
    if len(token_unembed_to_remove) > 0:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features", "cache_removed_token_unembed"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    else:
        fig = px.line(attn_df, 
                    y=["original_attn","reconstructed_attn", "ablated_feature_attn", "cache_only_features"],
                    hover_name="str_tokens", 
                    hover_data=["pos", "batch", "label"], 
                    title="Original vs Reconstructed attention")
    
    # increase figure height
    fig.update_layout(height=800)
    fig.show()


random_tokens, random_token_groups = generate_random_token_prompt(model, n_random_tokens=4, n_repeat_tokens=2)
example_prompt = model.to_string(random_tokens[:-1])
example_prompt_answer = model.to_string(random_tokens[:-1])
attn_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([example_prompt], model, sparse_autoencoder)
attn_df.tail(2).T
test_get_attn_sae_feature_removal(
    example_prompt, 
    example_prompt_answer, model, 
    sparse_autoencoder, features_to_remove=[773, 134,560])


torch.Size([1, 10, 8, 64])


# Directly Search for Positional Features

In [ ]:
# how can we automatically match features to positions?
# for position in position:
#     for feature in features: -> get recall and precision

positions = list(range(128)) # for the first 10 positions
features = list(range(sparse_autoencoder.cfg.d_sae)) # check all features

all_scores = []

for position in tqdm(positions):
    for feature in features:
        mask = [i  % 128 == position  for i in range(all_token_df.shape[0])]
        score = all_token_features[mask, feature].mean().item()
        all_scores.append([position, feature, score])
        
        
scores_df = pd.DataFrame(all_scores, columns=["pos", "feature", "score"])


In [ ]:
# get the top positiion for each feature
# get the top score per feature (criteria: activates at that pos)
top_feature_pos_scores_df = scores_df.groupby(["feature","pos"]).score.mean().reset_index().sort_values("score", ascending=False).groupby("feature").head(1).sort_values("pos")
# then for each pos, get the features who's maximal top score is at that position (criteria: activates more than anything else)
top_feature_pos_scores_df = top_feature_pos_scores_df.sort_values("score", ascending=False).groupby("pos").head(3).sort_values("pos")
top_feature_pos_scores_df.head()

In [ ]:
tmp = scores_df.pivot(index="pos", columns="feature", values="score")
print(tmp.shape)
px.imshow(tmp, color_continuous_scale="RdBu_r", title="Feature Scores by Position").show()
tmp = pd.DataFrame(
    tmp.loc[:,top_feature_pos_scores_df.feature.values].values,
    index = list(tmp.index.to_flat_index()),
    columns = [str(i) for i in list(top_feature_pos_scores_df.feature)],
)
px.imshow(tmp.T, color_continuous_scale="RdBu_r", title="Feature Scores by Position").show()

In [ ]:
px.line(tmp[["248", "560"]]).show()

In [ ]:
px.line(tmp)
# px.line(tmp / tmp.values.max(axis=0, keepdims=True))

In [ ]:
# px.line(tmp[["430"]]).show()
px.line(tmp[["573"]]).show()
px.line(tmp[["545"]]).show()

# Clustering

In [ ]:
from scipy.cluster import hierarchy

def plot_cosine_similarity_heatmap(df, restricted_labels, reorder = False, title="Pairwise Cosine Similarity Heatmap"):
    data_array = df.to_numpy()
    linkage = hierarchy.linkage(data_array)
    dendrogram = hierarchy.dendrogram(
        linkage, no_plot=True, color_threshold=-np.inf
    )
    if reorder:
        reordered_ind = dendrogram["leaves"]
        # reorder df by ind
        df = df.iloc[reordered_ind, reordered_ind]
        # data_array = df.to_numpy()

    # plot the cosine similarity matrix
    fig  = px.imshow(
            df.values,
            color_continuous_scale="RdBu",
            color_continuous_midpoint=0.0,
            labels={"color": "Cosine Similarity"},
        )
    fig.update_xaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )
    fig.update_yaxes(
        tickmode="array",
        tickvals=list(range(len(restricted_labels))),
        ticktext=restricted_labels,
        showgrid=False,
    )

    # don't show axes if there are more than 20 rows 
    if df.shape[0] > 20:
        fig.update_xaxes(
            visible=False,
        )
        fig.update_yaxes(
            visible=False,
        )
    return fig

feature_dirs = pd.DataFrame(sparse_autoencoder.W_dec.detach().cpu()).reset_index(drop=True).T.corr()
plot_cosine_similarity_heatmap(feature_dirs, restricted_labels=[i for i in range(sparse_autoencoder.cfg.d_sae)], reorder =True).show()

## what about non-positional features?

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=11)
tmp = top_feature_pos_scores_df.sort_values("score", ascending=True).head(256)
tmp["mean_activation"] = tmp.feature.apply(lambda x: all_token_features[:,x].mean().item())
clusterer.fit(tmp[["score", "mean_activation"]])
tmp["group"] = [f"group_{i}" for i in clusterer.labels_]
px.scatter(tmp, x="score", y="mean_activation", color="group",
           hover_data=["feature"])

In [ ]:
filter_columns = ["pos", "unique_token", "context", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence"]
for feature in torch.tensor([9, 545, 42]):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
    display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(10))

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=8,
    min_dist=0.15,
    n_components=2,
    metric="cosine",
    random_state=42,
)

clusterer = hdbscan.HDBSCAN(min_cluster_size=4)


ummap_result = reducer.fit_transform(feature_acts.T.detach().cpu())
# ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec.detach().cpu())
clusterer.fit(ummap_result)
print(clusterer.labels_)


fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=[str(i) for i in clusterer.labels_],
        hover_data = [[f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)]],
        opacity=0.5,
        template="plotly",
    )

# make points larger
fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()

In [ ]:
# set pandas col width limit
pd.set_option('display.max_colwidth', 200)

groups = pd.DataFrame(ummap_result, columns=["x","y"])
groups["group"] = [f"{i}" for i in clusterer.labels_]
groups["feature"] = [f"{i}" for i in range(sparse_autoencoder.cfg.d_sae)]
groups = groups[["feature", "group", "x", "y"]]


feature_group = "41"
features = groups[groups.group == feature_group].feature.astype(int).values
for feature in torch.tensor(features):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=3,
    min_dist=0.09,
    n_components=2,
    metric="euclidean",
    random_state=42,
)

clusterer = hdbscan.HDBSCAN(min_cluster_size=5)


ummap_result = reducer.fit_transform(sparse_autoencoder.W_dec.detach().cpu())
clusterer.fit(ummap_result)
# print(clusterer.labels_)


fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=[str(i) for i in clusterer.labels_],
        hover_data = [[f"feature_{i}" for i in range(sparse_autoencoder.cfg.d_sae)]],
        opacity=0.5,
        template="plotly",
    )

# make points larger
fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()

In [ ]:
# set pandas col width limit
pd.set_option('display.max_colwidth', 200)

groups = pd.DataFrame(ummap_result, columns=["x","y"])
groups["group"] = [f"{i}" for i in clusterer.labels_]
groups["feature"] = [f"{i}" for i in range(sparse_autoencoder.cfg.d_sae)]
groups = groups[["feature", "group", "x", "y"]]


feature_group = "37"
features = groups[groups.group == feature_group].feature.astype(int).values
for feature in torch.tensor(features):
    all_token_df[f"feature_{feature}"] = all_token_features[:,feature].detach().cpu()
    mean_activation = (all_token_df[f"feature_{feature}"] > 0).mean()
    if mean_activation > 0.001:
        print(feature.item(), (all_token_df[f"feature_{feature}"] > 0).mean()) # super super dense feature.
        display(all_token_df.sort_values(f"feature_{feature}", ascending=False)[filter_columns + [f"feature_{feature}"]].head(5))
    else:
        print(f"feature {feature.item()} is dead")